#  vLLM部署function call 测试

In [1]:
import json

from openai import OpenAI

In [2]:
def get_current_temperature(location: str, unit: str = "celsius"):
    """Get current temperature at a location.

    Args:
        location: The location to get the temperature for, in the format "City, State, Country".
        unit: The unit to return the temperature in. Defaults to "celsius". (choices: ["celsius", "fahrenheit"])

    Returns:
        the temperature, the location, and the unit in a dict
    """
    return {
        "temperature": 26.1,
        "location": location,
        "unit": unit,
    }


def get_temperature_date(location: str, date: str, unit: str = "celsius"):
    """Get temperature at a location and date.

    Args:
        location: The location to get the temperature for, in the format "City, State, Country".
        date: The date to get the temperature for, in the format "Year-Month-Day".
        unit: The unit to return the temperature in. Defaults to "celsius". (choices: ["celsius", "fahrenheit"])

    Returns:
        the temperature, the location, the date and the unit in a dict
    """
    return {
        "temperature": 25.9,
        "location": location,
        "date": date,
        "unit": unit,
    }


def get_function_by_name(name):
    if name == "get_current_temperature":
        return get_current_temperature
    if name == "get_temperature_date":
        return get_temperature_date

In [3]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_temperature",
      "description": "Get current temperature at a location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location to get the temperature for, in the format \"City, State, Country\"."
          },
          "unit": {
            "type": "string",
            "enum": [
              "celsius",
              "fahrenheit"
            ],
            "description": "The unit to return the temperature in. Defaults to \"celsius\"."
          }
        },
        "required": [
          "location"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_temperature_date",
      "description": "Get temperature at a location and date.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location to get the temperature for, in the format \"City, State, Country\"."
          },
          "date": {
            "type": "string",
            "description": "The date to get the temperature for, in the format \"Year-Month-Day\"."
          },
          "unit": {
            "type": "string",
            "enum": [
              "celsius",
              "fahrenheit"
            ],
            "description": "The unit to return the temperature in. Defaults to \"celsius\"."
          }
        },
        "required": [
          "location",
          "date"
        ]
      }
    }
  }
]


In [83]:
# 不使用慢思考
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2024-09-30"},
    {"role": "user",  "content": "What's the temperature in San Francisco now? How about tomorrow? /no_think"}
]

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2024-09-30"},
    {"role": "user",  "content": "What's the temperature in San Francisco now? /no_think"}
]

# 慢思考
# messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2024-09-30"},
#     {"role": "user",  "content": "What's the temperature in San Francisco now? How about tomorrow?"}
# ]

In [84]:
functions = [tool["function"] for tool in tools]


In [85]:
openai_api_key = "EMPTY"
openai_api_base = "http://192.168.11.65:18083/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

model_name = "qwen3-8b"

In [102]:
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    temperature=0.7,
    top_p=0.8,
    max_tokens=512,
    extra_body={
        "repetition_penalty": 1.05,
    },
    stream=True
)

In [103]:
# for chunk in response:
#     #print(chunk.choices[0].delta.content or "", end="")
#     content = chunk.choices[0]
    
#     print(content)


In [104]:
def extract_reasoning_and_calls(chunks: list):
    reasoning_content = ""
    tool_call_idx = -1
    arguments = []
    function_names = []
    tool_call_id =[]
    
    for chunk in chunks:
        if chunk.choices[0].delta.tool_calls:
            tool_call = chunk.choices[0].delta.tool_calls[0]
            if tool_call.index != tool_call_idx:
                tool_call_idx = chunk.choices[0].delta.tool_calls[0].index
                arguments.append("")
                function_names.append("")

            if tool_call.id:
                tool_call_id.append(tool_call.id)
                
            if tool_call.function:
                if tool_call.function.name:
                    function_names[tool_call_idx] = tool_call.function.name

                if tool_call.function.arguments:
                    arguments[tool_call_idx] += tool_call.function.arguments
        else:
            if hasattr(chunk.choices[0].delta, "reasoning_content"):
                reasoning_content += chunk.choices[0].delta.reasoning_content
    return tool_call_id, function_names, arguments, reasoning_content

In [105]:
tool_call_id, function_names, arguments, reasoning_content =extract_reasoning_and_calls(response)
tool_call_id, function_names, arguments, reasoning_content

(['chatcmpl-tool-c8c13129ffdd412680c8e6074747b423'],
 ['get_current_temperature'],
 ['{"location": "San Francisco, California, USA"}'],
 '')

In [87]:
tool_name = []
tool_id = []
arguments = []
tool_call_idx = -1
for chunk in response:
    content = chunk.choices[0]
    
    if chunk.choices[0].delta.tool_calls:
        tool_call = chunk.choices[0].delta.tool_calls[0]
        if tool_call.index != tool_call_idx:
            # if tool_call_idx >= 0:
            #     print(f"streamed tool call arguments: "
            #           f"{arguments[tool_call_idx]}")
            tool_call_idx = chunk.choices[0].delta.tool_calls[0].index
            arguments.append("")
        if tool_call.id:
            tool_id.append(tool_call.id)

        if tool_call.function:
            if tool_call.function.name:
                tool_name.append(tool_call.function.name)

            if tool_call.function.arguments:
                arguments[tool_call_idx] += tool_call.function.arguments
    
    if content.finish_reason == "tool_calls":
        
        for tool, id, argument in zip(tool_name, tool_id, arguments):
            print(tool_name1, tool_id1, arguments1)
        # for i in range(len(tool_name)):
        #     print(tool_name[i])
        #     print(tool_id[i])
        #     print(arguments[i])

get_temperature_date chatcmpl-tool-63996bd776ad4aee901e775132b1fd3e {"location": "San Francisco, California, USA", "date": "2024-09-31"}


In [74]:
arguments, tool_name, tool_id

(['{"location": "San Francisco, California, USA"}',
  '{"location": "San Francisco, California, USA", "date": "2024-09-31"}'],
 ['get_current_temperature', 'get_temperature_date'],
 ['chatcmpl-tool-fcfb0eec5d6649169641b52c839ace67',
  'chatcmpl-tool-ced59f49f6414dd68298cbe6fb099229'])

In [ ]:
messages.append(response.choices[0].message.model_dump())

In [ ]:
messages

In [ ]:
if tool_calls := response.choices[0].message.model_dump().get("tool_calls", None):
    for tool_call in tool_calls:
        fn_name = tool_call["function"]["name"]
        fn_args = json.loads(tool_call["function"]["arguments"])
        
        # 假设有函数库 get_function_by_name
        fn_result = json.dumps(get_function_by_name(fn_name)(**fn_args))
        
        messages.append({
            "role": "tool",
            "content": fn_result,
            "tool_call_id": tool_call["id"]
        })

In [ ]:
messages

In [ ]:
messagess = [
    {'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2024-09-30'},
    {'role': 'user', 'content': "What's the temperature in San Francisco now? How about tomorrow?"},
    {'role': 'assistant', 'content': None, 'function_call': None, 'tool_calls': [
        {'id': 'chatcmpl-tool-924d705adb044ff88e0ef3afdd155f15', 'function': {'arguments': '{"location": "San Francisco, CA, USA"}', 'name': 'get_current_temperature'}, 'type': 'function'},
        {'id': 'chatcmpl-tool-7e30313081944b11b6e5ebfd02e8e501', 'function': {'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-10-01"}', 'name': 'get_temperature_date'}, 'type': 'function'},
    ]},
    {'role': 'tool', 'content': '{"temperature": 26.1, "location": "San Francisco, CA, USA", "unit": "celsius"}', 'tool_call_id': 'chatcmpl-tool-924d705adb044ff88e0ef3afdd155f15'},
    {'role': 'tool', 'content': '{"temperature": 25.9, "location": "San Francisco, CA, USA", "date": "2024-10-01", "unit": "celsius"}', 'tool_call_id': 'chatcmpl-tool-7e30313081944b11b6e5ebfd02e8e501'},
]

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2024-09-30'},
    {'role': 'user', 'content': "What's the temperature in San Francisco now? How about tomorrow?"},
    {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [
        {'id': 'chatcmpl-tool-1ddfe7ec00254f39a3a21ab5ecc6756e', 'function': {'arguments': '{"location": "San Francisco, CA, USA"}', 'name': 'get_current_temperature'}, 'type': 'function'},
        {'id': 'chatcmpl-tool-cc553e7bf9ae401a851cf1b9d5901e72', 'function': {'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-10-01"}', 'name': 'get_temperature_date'}, 'type': 'function'}
    ]},
    {'role': 'tool', 'content': '{"temperature": 26.1, "location": "San Francisco, CA, USA", "unit": "celsius"}', 'tool_call_id': 'chatcmpl-tool-1ddfe7ec00254f39a3a21ab5ecc6756e'},
    {'role': 'tool', 'content': '{"temperature": 25.9, "location": "San Francisco, CA, USA", "date": "2024-10-01", "unit": "celsius"} /think', 'tool_call_id': 'chatcmpl-tool-cc553e7bf9ae401a851cf1b9d5901e72'}
]

In [ ]:
messages.append([{'role': 'user', 'content': 'what\'s your name'}])
messages

In [ ]:
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    temperature=0.7,
    top_p=0.8,
    max_tokens=512,
    extra_body={
        "repetition_penalty": 1.05,
    },
)



In [ ]:
messages.append(response.choices[0].message.model_dump())

In [ ]:
messages